In [ ]:
from rasterfoundry.api import API
refresh_token = '<Refresh Token>'
api = API(refresh_token=refresh_token)

# Getting analyses from the client

In [ ]:
analyses = api.analyses
analysis = analyses[0]
analysis

# Visualizing analyses

## Comparing to projects

In [ ]:
m = analysis.get_map()

projects = api.projects
project = projects[0]
project.compare(analysis, m)
m

## Exporting analyses using tile layers

In [ ]:
from ipyleaflet import DrawControl
dc = DrawControl()

m = analysis.get_map()
m.add_control(dc)
analysis.add_to(m)
m

# Draw a polygon on the map below

In [ ]:
last_draw = dc.last_draw
last_draw

### Calculate a bounding box from the last draw

In [ ]:
def snap_to_360(x_coord):
    """Snap an x coordinate to [-180, 180]
    
    Coordinates coming back from the API for some projects can be
    outside this range, and coordinates in the bbox outside this
    range make the export API upset. When it's upset, it returns
    an array with just a single 0 in it, which is not an accurate
    representation of the project normally.
    """
    return x_coord - round((x_coord + 180) / 360, 0) * 360

def geom_to_bbox(geom):
    coords = geom['geometry']['coordinates'][0]
    min_x = snap_to_360(min([point[0] for point in coords]))
    min_y = min([point[1] for point in coords])
    max_x = snap_to_360(max([point[0] for point in coords]))
    max_y = max([point[1] for point in coords])
    return ','.join(map(str, [min_x, min_y, max_x, max_y]))

In [ ]:
bbox = geom_to_bbox(last_draw)
bbox

### Request a synchronous export for our area 

In [ ]:
resp = analysis.get_export(bbox=bbox)
resp

### Visualize the export

If you don't have rasterio or matplotlib installed, you can run the cell at the bottom of this notebook

In [ ]:
from rasterio.io import MemoryFile
import matplotlib.pyplot as plt

data = resp.content
with MemoryFile(data) as memfile:
    with memfile.open() as dataset:
        plt.imshow(dataset.read(1), cmap='Purples')

In [ ]:
%%bash
pip install numpy matplotlib rasterio==1.0a12